In [ ]:
import pandas as pd

subscription_df = pd.read_excel("Subscription.xlsx", sheet_name="Sheet1") # Or your specific sheet name
bank_account_df = pd.read_excel("BankAccount.xlsx", sheet_name="Sheet1") # Or your specific sheet name

merged_df = pd.merge(
    bank_account_df,
    subscription_df[['investor_profile_id_raw', 'profile', 'routing', 'bank #']],
    left_on='investor_name',
    right_on='investor_profile_id_raw',
    how='left'
)

if 'investor_profile_id_raw' in merged_df.columns:
    merged_df = merged_df.drop(columns=['investor_profile_id_raw'])
else:
    print("Column 'investor_profile_id_raw' not found in merged_df. It might have been automatically handled or named differently.")

output_excel_filename = "Updated_BankAccount_Data.xlsx"
merged_df.to_excel(output_excel_filename, index=False, sheet_name="MergedData")
print(f"\nSuccessfully saved the updated table to '{output_excel_filename}'")

KeyError: 'investor_name'

这个bankaccount的权限我没有所以还没改到bankaccount中
join 的时候

join之后 按照subscription 中的 profile, routing, bank # 去重的结果
和按照原bankaccount中的 bank_account_transfer_type, bank_account_routing_number,	bank_account_account_number去重的结果不一样


In [25]:
import pandas as pd

# Step 1: 读取 BankAccount 的所有数据
bank_raw = pd.read_excel("BankAccount.xlsx", sheet_name="Sheet1", header=None)

# Step 2: 取最后一行为真正的列名
bank_headers = bank_raw.iloc[-1]
bank_account_df = bank_raw.iloc[:-1].copy()
bank_account_df.columns = bank_headers

# Step 3: 读取 Subscription.xlsx（正常结构）
subscription_df = pd.read_excel("Subscription.xlsx", sheet_name="Sheet1")

# Step 4: 合并两个数据集
merged_df = pd.merge(
    bank_account_df,
    subscription_df[['profile', 'routing', 'bank #']],
    left_on='investor_name',
    right_on='profile',
    how='left'
)

# Step 5: 删除多余的列
# if 'investor_profile_id_raw' in merged_df.columns:
#     merged_df = merged_df.drop(columns=['investor_profile_id_raw'])

#去重
# merged_df = merged_df.drop_duplicates(
#     subset=['profile', 'routing', 'bank #'], keep='first'
# )

# 去重：每个唯一银行账户只保留一次
# merged_df = merged_df.drop_duplicates(
#     subset=[
#         'profile',
#         'bank_account_transfer_type',
#         'bank_account_routing_number',
#         'bank_account_account_number'
#     ],
#     keep='first'
# )

# 统计每个 profile 拥有多少个唯一银行账户
# account_count = merged_df.groupby('profile')[
#     ['bank_account_transfer_type', 'bank_account_routing_number', 'bank_account_account_number']
# ].nunique()

# # 保留拥有多个唯一银行账户的 profile
# multiple_accounts = account_count[
#     (account_count['bank_account_transfer_type'] > 1) |
#     (account_count['bank_account_routing_number'] > 1) |
#     (account_count['bank_account_account_number'] > 1)
# ]

# # Step 6: 输出文件
output_excel_filename = "Updated_BankAccount_Data.xlsx"
merged_df.to_excel(output_excel_filename, index=False, sheet_name="MergedData")

# print(f"✅ Successfully saved the updated table to '{output_excel_filename}'")


In [22]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill

# 读取文件
file_path = "2-Updated_BankAccount_Data.xlsx"

df = pd.read_excel(file_path)

# 生成唯一银行账户标识键
df['account_key'] = (
    df['bank_account_transfer_type'].astype(str) + "|" +
    df['bank_account_routing_number'].astype(str) + "|" +
    df['bank_account_account_number'].astype(str)
)

# 去重用于分析（但不用于最终高亮）
dedup_df = df.drop_duplicates(
    subset=['profile', 'bank_account_transfer_type', 'bank_account_routing_number', 'bank_account_account_number']
)

# 多账户的 profile（一个 profile → 多个账户）
profile_counts = dedup_df.groupby('profile').size().reset_index(name='count')
multi_account_profiles = set(profile_counts[profile_counts['count'] > 1]['profile'])

# 多 profile 使用同一账户（一个账户 → 多个 profile）
account_counts = dedup_df.groupby('account_key')['profile'].nunique().reset_index(name='count')
shared_accounts = set(account_counts[account_counts['count'] > 1]['account_key'])

# 打开原始 Excel 文件进行高亮
wb = load_workbook(file_path)
ws = wb.active

# 获取列索引
header = [cell.value for cell in next(ws.iter_rows(min_row=1, max_row=1))]
profile_col = header.index('profile') + 1
transfer_col = header.index('bank_account_transfer_type') + 1
routing_col = header.index('bank_account_routing_number') + 1
number_col = header.index('bank_account_account_number') + 1

# 高亮样式
profile_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")  # 黄色
account_fill = PatternFill(start_color="00B0F0", end_color="00B0F0", fill_type="solid")  # 蓝色

# 从第二行开始遍历（数据行）
for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
    profile_val = row[profile_col - 1].value
    transfer = str(row[transfer_col - 1].value)
    routing = str(row[routing_col - 1].value)
    number = str(row[number_col - 1].value)
    account_key = f"{transfer}|{routing}|{number}"

    # 高亮 profile（黄色）
    if profile_val in multi_account_profiles:
        row[profile_col - 1].fill = profile_fill

    # 高亮银行账户字段（蓝色）
    if account_key in shared_accounts:
        row[transfer_col - 1].fill = account_fill
        row[routing_col - 1].fill = account_fill
        row[number_col - 1].fill = account_fill

# 保存高亮后的文件
highlighted_file_path = "Updated_BankAccount_Highlighted.xlsx"
wb.save(highlighted_file_path)

highlighted_file_path


'Updated_BankAccount_Highlighted.xlsx'

In [ ]:
import pandas as pd

# 读取合并后的表格
df = pd.read_excel("Updated_BankAccount_Data.xlsx")

# 去重：确保银行账户唯一组合不重复计数
unique_accounts_df = df.drop_duplicates(
    subset=['profile', 'bank_account_transfer_type', 'bank_account_routing_number', 'bank_account_account_number']
)

# 每个 profile 拥有多少个唯一银行账户
profile_to_account_count = unique_accounts_df.groupby('profile').size().reset_index(name='unique_bank_accounts')

# 找出拥有多个账户的 profile（正常情况）
multi_accounts = profile_to_account_count[profile_to_account_count['unique_bank_accounts'] > 1]

print("✅ 以下 profiles 拥有多个唯一银行账户：")

multi_accounts.to_excel("Profiles_With_Multiple_BankAccounts.xlsx", index=False)


In [ ]:
# 去重：确保组合唯一
unique_accounts_df = df.drop_duplicates(
    subset=['profile', 'bank_account_transfer_type', 'bank_account_routing_number', 'bank_account_account_number']
)

# 构建账户标识键（方便分组）
unique_accounts_df['account_key'] = (
    unique_accounts_df['bank_account_transfer_type'].astype(str) + "|" +
    unique_accounts_df['bank_account_routing_number'].astype(str) + "|" +
    unique_accounts_df['bank_account_account_number'].astype(str)
)

# 每个账户 key 对应多少个不同的 profile
account_to_profile_count = unique_accounts_df.groupby('account_key')['profile'].nunique().reset_index(name='profile_count')

# 找出被多个 profile 共享的银行账户
shared_accounts = account_to_profile_count[account_to_profile_count['profile_count'] > 1]

print("⚠️ 以下银行账户被多个 profiles 使用（可能是夫妻共享账户等情况）：")
print(shared_accounts)

shared_accounts.to_excel("BankAccounts_Shared_By_Multiple_Profiles.xlsx", index=False)
